In [ ]:
import os
from glob import glob
from copy import deepcopy

import numpy as np
import matplotlib.pyplot as plt

import pydicom
from pymedphys.type import DeliveryData
from pymedphys.dcm import anonymise_dicom, coords_and_dose_from_dcm
from pymedphys.coll import find_relevant_control_points
from pymedphys.logfile import delivery_data_from_logfile
from pymedphys.gamma import gamma_dcm, convert_to_percent_pass

from pymedphys.plt import pcolormesh_grid

In [ ]:
data_root = r'C:\Users\sbiggs\logfile2dicom'

In [ ]:
logfile_dose_dicom_path = glob(os.path.join(
    data_root, 'calced_dose', '*Logfile*.dcm'
))[0]

logfile_dose_dicom_path

In [ ]:
patient_dose_dicom_path = glob(os.path.join(
    data_root, 'calced_dose', '*BREAST*.dcm'
))[0]

patient_dose_dicom_path

In [ ]:
coords_reference, dose_reference = coords_and_dose_from_dcm(
    patient_dose_dicom_path)
coords_evaluation, dose_evaluation = coords_and_dose_from_dcm(
    logfile_dose_dicom_path)

In [ ]:
load_filepath = os.path.join(
    data_root, 'gamma_calcs_2percent.npz'
)

In [ ]:
gamma_load = np.load(load_filepath)

In [ ]:
gamma = gamma_load['2']

In [ ]:
distance_threshold = 2

In [ ]:
valid_gamma = gamma[~np.isnan(gamma)]

num_points = len(valid_gamma)
weights = np.ones(num_points)/num_points

plt.hist(valid_gamma, 20, weights=weights)
plt.title('Gamma histogram 2% / {} mm | 20% lower dose cutoff'.format(distance_threshold))
plt.xlabel('Local Gamma')
plt.ylabel('Normalised bin count')

max_x_axis = np.ceil(np.max(valid_gamma))

plt.xlim([0, max_x_axis])
plt.show()

In [ ]:
np.mean(valid_gamma<1)

In [ ]:
max_gamma = np.max(valid_gamma)
indices = np.where(gamma == max_gamma)
lookup = [coord[index] for coord, index in zip(coords_reference, indices)]
lookup

In [ ]:
indices

In [ ]:
x, y = pcolormesh_grid(coords_reference[0], coords_reference[1])
plt.pcolormesh(x, y, gamma[:,:,54].T, vmin=0, vmax=2, cmap='coolwarm')
plt.axis('equal')
plt.xlim([-400,-200])
plt.ylim([-20,200])
plt.colorbar()

In [ ]:
np.nanstd(diff / dose_reference)

In [ ]:
diff = dose_evaluation - dose_reference
max_diff = np.max(np.abs(diff))
max_dose = np.max([np.max(dose_evaluation), np.max(dose_reference)])

i = 54

nanned_dose_evaluation == 0


x, y = pcolormesh_grid(coords_reference[0], coords_reference[1])

plt.pcolormesh(x, y, dose_reference[:,:,i].T, vmin=0, vmax=max_dose)
plt.axis('equal')
plt.xlim([-400,-200])
plt.ylim([-20,200])
plt.colorbar()
plt.show()

plt.pcolormesh(x, y, dose_evaluation[:,:,i].T, vmin=0, vmax=max_dose)
plt.axis('equal')
plt.xlim([-400,-200])
plt.ylim([-20,200])
plt.colorbar()
plt.show()

plt.pcolormesh(x, y, diff[:,:,i].T, vmin=-max_diff, vmax=max_diff, cmap='bwr')
plt.axis('equal')
plt.xlim([-400,-200])
plt.ylim([-20,200])
plt.colorbar()
plt.show()


In [ ]:
coords_evaluation

In [ ]:
coords_reference